In [1]:
import sys 
#sys.path.append('/home/monte.flora/python_packages/wofs_ml_severe')
sys.path.append('/home/samuel.varga/python_packages/WoF_post')
#sys.path.append('/home/samuel.varga/python_packages/wofs_ml_severe')
sys.path.append('/home/samuel.varga/projects/2to6_hr_severe_wx/experiments')
sys.path.append('/home/samuel.varga/python_packages/MontePython/')
#from ml_2to6_data_pipeline import (GridPointExtracter,
#                                                       subsampler, 
#                                                       load_dataset)
from ml_2to6_data_pipeline import (GridPointExtracter,
                                                       subsampler, 
                                                       load_dataset, random_subsampler)
from os.path import join, exists

ModuleNotFoundError: No module named 'importlib_resources'

In [7]:
#dates = ['20180504',
#       '20180509', '20180512', '20180519', '20180524', '20180530',
#       '20190513', '20190518', '20190521', '20190522', '20200520',
#       '20200521', '20200526', '20200528', '20210506', '20210510',
#       '20210513', '20210514', '20210521', '20210523', '20210524',
#       '20210525']
FRAMEWORK='ADAM'
TIMESCALE='0to3'
dates=[
  '20180524'
       ]

In [8]:
##date = '20210504'

if TIMESCALE=='0to3':
    init_time='0000'#Predictions valid from 0000-0300
else:
    init_time='2200' #Predictions valid from 0000-0400 this would actually be valid the day after, so change this

    #init_time = '1900' #3 hours out from effective time? Should be same init time in storm report?

for date in dates:
    #path = f'/work/samuel.varga/data/2to6_hr_severe_wx/SummaryFiles/{date}/{init_time}'
    path = f'/work/mflora/SummaryFiles/{date}/{init_time}' #What files would I need to make it myself? Env and storm variables?
    out_path = join(f'/work/samuel.varga/data/{TIMESCALE}_hr_severe_wx/{FRAMEWORK}', 'Single') 

    X_env, X_strm, ncfile, llgrid  = load_dataset(path, TIMESCALE=TIMESCALE)
    extracter = GridPointExtracter(ncfile, env_vars=X_env.keys(), strm_vars=X_strm.keys(), ll_grid=llgrid, TIMESCALE=TIMESCALE, FRAMEWORK=FRAMEWORK)
    df = extracter(X_env, X_strm, predict=True) #True 
    df.reset_index(drop=True, inplace=True)
    print(len(df))
    
    out_name = join(out_path, f'wofs_ml_severe__{TIMESCALE}hr__{date}_data.feather')
    print(f'Saving {out_name}...')
    df.to_feather(out_name)

True and ADAM
False and ADAM
dict_keys(['uh_2to5_instant__time_max__9km__smoothed'])
dict_keys(['uh_2to5_instant__time_max__4km', 'uh_0to2_instant__time_max__4km', 'wz_0to2_instant__time_max__4km', 'comp_dz__time_max__4km', 'ws_80__time_max__4km', 'hailcast__time_max__4km', 'w_up__time_max__4km', 'okubo_weiss__time_max__4km'])
7056
Saving /work/samuel.varga/data/0to3_hr_severe_wx/ADAM/Single/wofs_ml_severe__0to3hr__20180524_data.feather...


# Trouble shooting steps

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
X=pd.read_feather('/work/samuel.varga/data/0to3_hr_severe_wx/ADAM/Single/wofs_ml_severe__0to3hr__20180512_data.feather')

In [16]:
import numpy as np
x=np.arange(0,10)

In [21]:
x[::None]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [16]:
FRAMEWORK=['POTVIN','ADAM']; TIMESCALE='2to6'; date='20180524'; init_time='0000'
base_path = f'/work/mflora/SummaryFiles/{date}/{init_time}'
SUMMARY_FILE_OUT_PATH = '/work/samuel.varga/data/{}_hr_severe_wx'.format(TIMESCALE) 
X_env, X_strm, ncfile, ll_grid  = load_dataset(base_path, TIMESCALE=TIMESCALE) #Load the files for the time scale
inds=None #set to none
    
for framework in FRAMEWORK:
        print(framework)
        extracter = GridPointExtracter(ncfile, env_vars=X_env.keys(), strm_vars=X_strm.keys(), ll_grid=ll_grid, TIMESCALE=TIMESCALE, FRAMEWORK=framework) #Def GPE-- pass timescale and framework through
        df = extracter(X_env, X_strm) #Apply GPE to the env and storm
        print(len(df))
        #ys = [f for f in df.columns if 'severe' in f]
        #y_df = df[ys].sum(axis='columns')

        # Sampling all grid points with an event, but only 15% of 
        # grid points with no events. 
        #inds = subsampler(y_df, pos_percent=1.0, neg_percent=1.0) #Loken et. didn't resample, so use 1

        if inds is None: #Inds will be none on the first call. For the second framework, inds will already be assigned
            inds = random_subsampler(len(df), percent=0.3)
        
        df_sub = df.iloc[inds, :] #Selects subset based on inds-- will choose the same indices for both frameworks
        df_sub.reset_index(drop=True, inplace=True)

        out_path = path.replace(base_path, join(SUMMARY_FILE_OUT_PATH, f'{framework}/Single')) #replace the base path with the output path
        if not exists(out_path):
            os.makedirs(out_path)

        out_name = join(out_path, 'wofs_ML{}.feather'.format(TIMESCALE.upper()))
        print(f'Saving {out_name}...')
        print(len(df_sub))
        df_sub.to_feather(out_name)

POTVIN
True and POTVIN
False and POTVIN
7056
Saving /work/samuel.varga/data/2to6_hr_severe_wx/POTVIN/Single/wofs_ML2TO6.feather...
2116
ADAM
True and ADAM
False and ADAM
dict_keys(['uh_2to5_instant__time_max__9km__smoothed'])
dict_keys(['uh_2to5_instant__time_max__4km', 'uh_0to2_instant__time_max__4km', 'wz_0to2_instant__time_max__4km', 'comp_dz__time_max__4km', 'ws_80__time_max__4km', 'hailcast__time_max__4km', 'w_up__time_max__4km', 'okubo_weiss__time_max__4km'])
7056
Saving /work/samuel.varga/data/2to6_hr_severe_wx/ADAM/Single/wofs_ML2TO6.feather...
2116


In [7]:
random_subsampler(150, percent=0.3)

array([ 25,   9, 125,  98,  44, 149,  41,  63, 116,  61, 143,   2, 127,
        94,  69,  86, 123, 111, 148, 147, 138,  68, 129, 139,  90, 122,
         3,   8,   0, 109,  45,  16,  76,  60,  50, 119,  74, 134, 126,
        33,  89,  10, 135, 108, 120])